In [169]:
import graph_tool.all as gt
import os
import numpy as np

In [170]:
np.random.geometric(0.5)

2

In [171]:
data_dict = {
    'offline': {
        "school_friendship": [
            "spanish_highschools__1",
            "spanish_highschools__2",
            "spanish_highschools__6",
            "spanish_highschools__11_10",
            "spanish_highschools__11_9",
        ],
        "school_contact": [
            "sp_high_school_new__2011",
            "sp_high_school_new__2012",
            "sp_primary_school",
        ],
        'email': [
            "email_enron",
            "uni_email",
            "email_eu",
            "dnc"
        ]
    },
    'online': {
        'facebook': [
            "facebook_wall",
            "facebook_organizations__L1",
            "facebook_organizations__L2",
            "facebook_organizations__M1",
            "ego_social__facebook_107",
            "ego_social__facebook_1912",
            "ego_social__facebook_combined",
        ],
        'google_plus': [
            "ego_social__gplus_101133961721621664586",
            "ego_social__gplus_100500197140377336562",
            "ego_social__gplus_101133961721621664586",
            "ego_social__gplus_114336431216099933033",
        ],
        'twitter': [
            "twitter_15m",
            "twitter",
        ],
    },
}

In [172]:
def duplicate_graph(g, k):
    """
    Creates a new graph by combining `k` disjoint copies of graph `g`.
    """
    combined = gt.Graph()
    vertex_maps = []

    for _ in range(k):
        g_copy = g.copy()
        iterator = combined.add_vertex(g_copy.num_vertices())
        node_map = {v: next(iterator) for v in g_copy.vertices()}
        # Add edges with correct offset
        for e in g_copy.edges():
            src = int(e.source())
            tgt = int(e.target())
            combined.add_edge(node_map[src], node_map[tgt])
    
    return combined

In [173]:
import graph_tool.all as gt
import random
def extract_subgraph(g, node_set):
    vfilt = g.new_vertex_property("bool")
    for v in node_set:
        vfilt[int(v)] = True  # convert Vertex to int (vertex index)
    gv = gt.GraphView(g, vfilt=vfilt)
    return gt.Graph(gv, prune=True)
    
def forest_fire_sample(g, sample_size=500, pf=0.5):
    """
    Forest Fire Sampling on a graph-tool Graph.
    - g: graph-tool Graph
    - sample_size: desired number of nodes in sample
    - pf: forward burning probability
    """

    visited = set()
    not_visited = set(g.vertices())
    to_visit = []

    ego_count = 0
    while len(visited) < sample_size:
        if not to_visit or ego_count > sample_size:
            to_visit = [np.random.choice(list(not_visited))]
            ego_count = 0
        
        current = to_visit.pop()
        if current in visited:
            ego_count += 1
            continue

        visited.add(current)
        not_visited.remove(current)

        neighbors = list(current.out_neighbors())
        scaled_num_neighbors = max(min(len(neighbors), 5), int(len(neighbors) * sample_size / g.num_vertices()))
        num_to_burn = min(scaled_num_neighbors, np.random.geometric(pf))
        num_samples = scaled_num_neighbors - num_to_burn
        neighbors_sample = random.sample(neighbors, num_samples)
        to_visit.extend(neighbors_sample)

    return extract_subgraph(g, visited)

def set_num_edges(g, num_edges):
    # remove random edges until the graph has the desired number of edges
    while g.num_edges() > num_edges:
        edge = random.choice(list(g.edges()))
        g.remove_edge(edge)
    return g
    
def summary(g):
    return ", \t ".join([
        f"nodes: {g.num_vertices():.2f}",
        f"edges: {g.num_edges():.2f}",
        f"avg_degree: {2 * g.num_edges() / g.num_vertices():.2f}",
        f"avg_clustering: {gt.global_clustering(g)[0]:.2f}",
    ])

In [174]:
def export_graph(g, full_name, g_type, path):
    seen = set()
    filepath = f"{path}/{g_type}/{full_name}.edge"
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    with open(filepath, "w") as f:
        for e in g.edges():
            u, v = int(e.source()), int(e.target())
            edge = tuple(sorted((u, v)))
            if edge not in seen:
                seen.add(edge)
                f.write(f"{edge[0]} {edge[1]}\n")

In [175]:
source_dir = os.path.join("..", "..", "DATA", "netzschleuder")
target_dir = os.path.join("..", "..", "DATA", "processed")

network_list = [net for off_on in data_dict.values() for category in off_on.values() for net in category]
# network_list = ['sp_high_school_new__2011']

SAMPLE_SIZE = 400
DO_SAMPLE_UP = True
SET_NUM_EDGES = True

samples = []
for network in network_list:
    # Load the graph
    g = gt.load_graph(f"{source_dir}/{network}.gt")

    # Get the number of vertices and edges
    num_nodes = g.num_vertices()
    num_edges = g.num_edges()

    g.set_directed(False)
    gt.remove_parallel_edges(g)
    gt.remove_self_loops(g)    

    # Print the number of vertices and edges
    print(f"\n Graph: {network}")
    print("\t Original: \t", summary(g))
    # export_graph(g, network, "original", target_dir)

    
    if num_nodes < SAMPLE_SIZE:
        if DO_SAMPLE_UP:
            g_new = duplicate_graph(g, int(np.ceil(SAMPLE_SIZE / num_nodes)))
            sample = forest_fire_sample(g_new, sample_size=SAMPLE_SIZE, pf=0.5)  
        else:
            sample = g
    else:
        sample = forest_fire_sample(g, sample_size=SAMPLE_SIZE, pf=0.5)    
        print("\t Sampled: \t", summary(sample))

    samples.append(sample)

min_edges = min([s.num_edges() for s in samples])
for network, sample in zip(network_list, samples):
    # reduce edges to the minimum
    if SET_NUM_EDGES:
        sample = set_num_edges(sample, min_edges)

    dir_name = f"sample_{"edged_" if SET_NUM_EDGES else ""}{"max_" if not DO_SAMPLE_UP else ""}{SAMPLE_SIZE}"
    export_graph(sample, network, dir_name, target_dir)





 Graph: spanish_highschools__1
	 Original: 	 nodes: 409.00, 	 edges: 6509.00, 	 avg_degree: 31.83, 	 avg_clustering: 0.34
	 Sampled: 	 nodes: 400.00, 	 edges: 6377.00, 	 avg_degree: 31.89, 	 avg_clustering: 0.34

 Graph: spanish_highschools__2
	 Original: 	 nodes: 238.00, 	 edges: 2889.00, 	 avg_degree: 24.28, 	 avg_clustering: 0.47

 Graph: spanish_highschools__6
	 Original: 	 nodes: 534.00, 	 edges: 9527.00, 	 avg_degree: 35.68, 	 avg_clustering: 0.45
	 Sampled: 	 nodes: 400.00, 	 edges: 5778.00, 	 avg_degree: 28.89, 	 avg_clustering: 0.45

 Graph: spanish_highschools__11_10
	 Original: 	 nodes: 458.00, 	 edges: 5866.00, 	 avg_degree: 25.62, 	 avg_clustering: 0.39
	 Sampled: 	 nodes: 400.00, 	 edges: 5081.00, 	 avg_degree: 25.41, 	 avg_clustering: 0.40

 Graph: spanish_highschools__11_9
	 Original: 	 nodes: 515.00, 	 edges: 5027.00, 	 avg_degree: 19.52, 	 avg_clustering: 0.34
	 Sampled: 	 nodes: 400.00, 	 edges: 3831.00, 	 avg_degree: 19.16, 	 avg_clustering: 0.36

 Graph: sp_high_s

In [176]:
# gt.graph_draw(g, output_size=(600, 600), bg_color=(255, 255, 255, 255))
# gt.graph_draw(sample, output_size=(600, 600), bg_color=(255, 255, 255, 255))
